In [ ]:
import random

# define the possible moves
moves = ['rock', 'paper', 'scissors']

# define a function to simulate a game of Rock Paper Scissors
def play(player1, player2):
    if player1 == player2:
        return 'draw'
    elif (player1 == 'rock' and player2 == 'scissors') or \
         (player1 == 'paper' and player2 == 'rock') or \
         (player1 == 'scissors' and player2 == 'paper'):
        return 'player1 wins'
    else:
        return 'player2 wins'

# define a function to generate a random move
def random_move():
    return random.choice(moves)

# define a function to simulate a game between two random players
def random_game():
    player1 = random_move()
    player2 = random_move()
    return play(player1, player2)

# define a function to simulate multiple games between two random players and calculate the win and draw ratios
def simulate(num_games):
    wins = 0
    draws = 0
    for i in range(num_games):
        result = random_game()
        if result == 'player1 wins':
            wins += 1
        elif result == 'draw':
            draws += 1
    win_ratio = wins / num_games
    draw_ratio = draws / num_games
    return win_ratio, draw_ratio

# run the simulation with 10000 games
win_ratio, draw_ratio = simulate(100)

# print the results
print('Win ratio:', win_ratio)
print('Draw ratio:', draw_ratio)


Win ratio: 0.38
Draw ratio: 0.21


In [ ]:
import random
from math import sqrt, log

class Node:
    def __init__(self, move=None, parent=None):
        self.move = move
        self.parent = parent
        self.children = []
        self.wins = 0
        self.visits = 0

    def select_child(self):
        return max(self.children, key=lambda c: c.wins / c.visits + sqrt(2 * log(self.visits) / c.visits))

    def expand_child(self, move):
        child = Node(move, self)
        self.children.append(child)
        return child

    def update(self, result):
        self.visits += 1
        self.wins += result

def simulate_game(agent1, agent2):
    moves = ['rock', 'paper', 'scissors']
    move1 = agent1.choose_move()
    move2 = agent2.choose_move()

    if move1 == move2:
        return 0  # Draw
    elif (move1 == 'rock' and move2 == 'scissors') or (move1 == 'paper' and move2 == 'rock') or (move1 == 'scissors' and move2 == 'paper'):
        return 1  # Agent 1 wins
    else:
        return -1  # Agent 2 wins

class Agent:
    def __init__(self):
        self.root = Node()
        self.current_node = self.root

    def choose_move(self):
        if len(self.current_node.children) == 0:
            return random.choice(['rock', 'paper', 'scissors'])

        child = self.current_node.select_child()
        self.current_node = child
        return child.move

    def update_tree(self, move):
        for child in self.current_node.children:
            if child.move == move:
                self.current_node = child
                return

        new_child = self.current_node.expand_child(move)
        self.current_node = new_child

    def backpropagate(self, result):
        node = self.current_node
        while node is not None:
            node.update(result)
            node = node.parent

def calculate_convergence_rate(draws, total_games):
    if total_games == 0:
        return 0

    return draws / total_games

def play_until_convergence():
    agent1 = Agent()
    agent2 = Agent()
    total_games = 0
    draws = 0

    while True:
        move1 = agent1.choose_move()
        move2 = agent2.choose_move()

        agent1.update_tree(move1)
        agent2.update_tree(move2)

        result = simulate_game(agent1, agent2)
        agent1.backpropagate(result)
        agent2.backpropagate(-result)

        if result == 0:
            draws += 1

        total_games += 1

        convergence_rate = calculate_convergence_rate(draws, total_games)
        if convergence_rate >= 0.95:
            break

    return convergence_rate, draws / total_games

convergence_rate, draw_rate = play_until_convergence()
print("Convergence Rate:", convergence_rate)
print("Draw Rate:", draw_rate)
